In [1]:
import numpy as np
import pandas as pd
from scipy.io import mmread
import os
import glob
import anndata
import tqdm
import zarr

- merge marker genes (types ABC)
- feed into NN design

In [2]:
rootpth = '/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/'
dirs = np.sort(glob.glob("*", root_dir=rootpth))

In [3]:
dirs

array(['P14_nr_1_a', 'P14_nr_1_b', 'P14_nr_2_a', 'P14_nr_2_b',
       'P17_nr_1_a', 'P17_nr_1_b', 'P17_nr_2_a', 'P17_nr_2_b',
       'P21_nr_1_a', 'P21_nr_1_b', 'P21_nr_2_a', 'P21_nr_2_b',
       'P28_dl_1_a', 'P28_dl_1_b', 'P28_dl_2_a', 'P28_dl_2_b',
       'P28_dr_1_a', 'P28_dr_1_b', 'P28_dr_3_a', 'P28_dr_3_b',
       'P28_nr_1_a', 'P28_nr_1_b', 'P28_nr_2_a', 'P28_nr_2_b',
       'P38_dr_1_a', 'P38_dr_1_b', 'P38_dr_2_a', 'P38_dr_2_b',
       'P38_nr_1_a', 'P38_nr_2_a', 'P38_nr_2_b', 'P8_nr_1_a', 'P8_nr_1_b',
       'P8_nr_2_a', 'P8_nr_2_b', 'processed'], dtype='<U10')

In [5]:
[d for d in dirs if d.startswith("P14")]

['P14_nr_1_a', 'P14_nr_1_b', 'P14_nr_2_a', 'P14_nr_2_b']

In [6]:
newdirs = [d for d in dirs if d.startswith("P14_nr")]
newdirs

['P14_nr_1_a', 'P14_nr_1_b', 'P14_nr_2_a', 'P14_nr_2_b']

In [7]:
# pth = '/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/P38_nr_1_a/filtered_feature_bc_matrix'
# !ls $pth

In [8]:
for i, _dir in tqdm.tqdm(enumerate(newdirs)):
    pth = os.path.join(rootpth, _dir, 'filtered_feature_bc_matrix')
    output = os.path.join(pth, 'counts.h5ad')
    print(pth)
    print(i, output)
    
    # check
    if os.path.isfile(output):
        print(_dir + " skipped...")
        continue
    
    # fnames 
    fc = os.path.join(pth, 'barcodes.tsv.gz')
    fg = os.path.join(pth, 'features.tsv.gz')
    fm = os.path.join(pth, 'matrix.mtx')
    fmgz = os.path.join(pth, 'matrix.mtx.gz')
    
    if not os.path.isfile(fc): 
        print(_dir + " failed...")
        continue
        
    if not os.path.isfile(fg): 
        print(_dir + " failed...")
        continue
        
    if not (os.path.isfile(fm) or os.path.isfile(fmgz)):
        print(_dir + " failed...")
        continue

    # load
    cells = pd.read_csv(fc, sep='\t', header=None)[0].values
    genes = pd.read_csv(fg, sep='\t', header=None).rename(columns={0: 'id', 1: 'name'})[['id', 'name']]
    if os.path.isfile(fm):
        mtx = mmread(fm)
    elif os.path.isfile(fmgz):
        mtx = mmread(fmgz)
    print(cells.shape, genes.shape, mtx.shape)
    
    # check
    if len(cells) != mtx.shape[1]:
        print(_dir + " failed...")
        continue
        
        
    # adata 
    adata = anndata.AnnData(
        X=mtx.T.tocsc(), 
        obs=pd.DataFrame(index=cells), 
        var=genes,
    )
    adata.write(output)

2it [00:00, 18.45it/s]

/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/P14_nr_1_a/filtered_feature_bc_matrix
0 /bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/P14_nr_1_a/filtered_feature_bc_matrix/counts.h5ad
P14_nr_1_a skipped...
/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/P14_nr_1_b/filtered_feature_bc_matrix
1 /bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/P14_nr_1_b/filtered_feature_bc_matrix/counts.h5ad
P14_nr_1_b skipped...
/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/P14_nr_2_a/filtered_feature_bc_matrix
2 /bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2022_Cell_v1ctx/P14_nr_2_a/filtered_feature_bc_matrix/counts.h5ad
P14_nr_2_a skipped...
/bigstore/GeneralStorage/fangming/projects/visctx/data_dump/counts/scRNAseq_Cheng2

4it [00:00, 17.58it/s]

P14_nr_2_b skipped...
